# Introduction

In this compition we are going to do prediction of house price.Prediction house prices are expected to help people who plan to buy a house so they can know the price range in the future, then they can plan their finance well. In addition, house price predictions are also beneficial for property investors to know the trend of housing prices in a certain location.
we are using advanced machine learning techniques for doing prediction.

# Contents

1. Importing important libraries
2. Reading dataset
3. Data visualization
4. Feature Engineering
5. Data pre-processing
6. Building models
7. Base models
8. Stacking models
9. Submission
10. Conclusion

# Importing important libraries...

In [ ]:
# here we are importing libraries for doing further analysis.
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from scipy import stats
from scipy.stats import norm, skew

# Reading dataset...

In [ ]:
Train = pd.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
Test = pd.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")

In [ ]:
# here we are printing first 5 lines of our train dataset
Train.head()

In [ ]:
# Here we are printing first 5 line of our test dataset
Test.head()

# Data visualization

In [ ]:
# here we are exploring outliers
fig, ax = plt.subplots()
ax.scatter(x = Train['GrLivArea'], y = Train['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GrLivArea', fontsize=13)
plt.show()

here we can see that at the bottom right two with extremely otliers

In [ ]:
# here we are deleting outliers
Train = Train.drop(Train[(Train['GrLivArea']>4000) & (Train['SalePrice']<300000)].index)

#Checking graphic again
fig, ax = plt.subplots()
ax.scatter(Train['GrLivArea'], Train['SalePrice'])
plt.ylabel('SalePrice', fontsize=13)
plt.xlabel('GrLivArea', fontsize=13)
plt.show()

Traget Variable.
SalePrice is the variable we need to predict, So let's do some analysis on this variable first.

In [ ]:
sns.distplot(Train['SalePrice'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(Train['SalePrice'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

#Plot the QQ-plot
fig = plt.figure()
res = stats.probplot(Train['SalePrice'], plot=plt)
plt.show()

here we are doing Log-transformation of the target variable.

In [ ]:
#We use the numpy fuction log1p which  applies log(1+x) to all elements of the column
Train["SalePrice"] = np.log1p(Train["SalePrice"])

#Checking the new distribution 
sns.distplot(Train['SalePrice'] , fit=norm);

# Get the fitted parameters used by the function
(mu, sigma) = norm.fit(Train['SalePrice'])
print( '\n mu = {:.2f} and sigma = {:.2f}\n'.format(mu, sigma))

#Now plot the distribution
plt.legend(['Normal dist. ($\mu=$ {:.2f} and $\sigma=$ {:.2f} )'.format(mu, sigma)],
            loc='best')
plt.ylabel('Frequency')
plt.title('SalePrice distribution')

#Plot the QQ-plot
fig = plt.figure()
res = stats.probplot(Train['SalePrice'], plot=plt)
plt.show()

# Feature Engineering...

In [ ]:
# here we are printing sum of all null values in train dataset
print(Train.isnull().sum())
print(sum(Train.isnull().sum()))

In [ ]:
# here we are printing sum of all null values in test dataset
print(Test.isnull().sum())
print(sum(Test.isnull().sum()))

In [ ]:
# here we are printing shape of our train and test dataset
print("Train shape:", Train.shape)
print("Test shape:", Test.shape)

In [ ]:
Train.info()


In [ ]:
Test.info()

In [ ]:
# Saving the 'Id' column
train_ID = Train['Id']
test_ID = Test['Id']
# now we are droping id column because this feature is not neccesary for prediction process
Train = Train.drop("Id",axis=1)
Test = Test.drop("Id",axis=1)

In [ ]:
# here we are again printing shape of our dataset to see that id column is replace or not
print("Train shape:", Train.shape)
print("Test shape:", Test.shape)

In [ ]:
# here we are printing all the name of columns that are present in our dataset
for col in Train.columns: 
    print(col) 

In [ ]:
# here we are printing summary of train dataset
Train.describe()

In [ ]:
# here we are printing summary of test dataset
Test.describe()

# Data pre-processing

In [ ]:
# here we are visualising null value in our train dataset
sns.heatmap(Train.isnull())

In [ ]:
# here we are visualising null value in our test dataset
sns.heatmap(Test.isnull())

In [ ]:
# here we are concating our train and test dataset
ntrain = Train.shape[0]
ntest = Test.shape[0]
y_train = Train.SalePrice.values
train_test_data = pd.concat((Train, Test)).reset_index(drop=True)


In [ ]:
# here we are droping salesprices columns from our concat dataset
train_test_data.drop(["SalePrice"],axis=1,inplace=True)


In [ ]:
# now we are printing shape of our concat dataset
print("train_test_data size is: {}".format(train_test_data.shape))

In [ ]:
# here we are counting the null value of each column
null = pd.DataFrame(train_test_data.isnull().sum().sort_values(ascending=False)[:25])
null.columns = ["Null counts"]
null.index.name = "Feature"
null

In [ ]:
# here we are counting null value in percent
all_data_na = (train_test_data.isnull().sum() / len(train_test_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head(20)

In [ ]:
# here we are plotting the upper tabular form into graph
plt.style.use(style='ggplot')
plt.rcParams['figure.figsize'] = (10, 6)
plt.xticks(rotation='90')
sns.barplot(x=all_data_na.index, y=all_data_na)
plt.xlabel('Features', fontsize=10)
plt.ylabel('Percent of missing values', fontsize=10)
plt.title('Percent missing data by feature', fontsize=13)

In [ ]:
# here we are making correlation graph to see how features are correlated with saleprice
plt.figure(figsize=(10,8))
corrMatrix=Train.corr()
sns.heatmap(corrMatrix,vmax=0.9,square =True)
plt.show()

**Imputing missing value**

In [ ]:
# firstly we are filling null value of column PoolQC
# data description says null means "No Pool". That make sense,
# given the huge ratio of missing value (+99%) and majority of houses have no Pool at all in general,
# so we are going to fill null values with none
train_test_data["PoolQC"] = train_test_data["PoolQC"].fillna("None")

In [ ]:
# secondly we are filling null values of column MiscFeature
# data description says null means "no misc feature",as given that the huge ratio of missing value,
# so we are going to fill null values with none
train_test_data["MiscFeature"] = train_test_data["MiscFeature"].fillna("None")

In [ ]:
# now we are filling null values of column Alley 
# data description says NA means "no alley access", as given that the huge ratio of missing value,
# so we are going to fill null values with none
train_test_data["Alley"] = train_test_data["Alley"].fillna("None")

In [ ]:
# now we are filling null values of column Fence 
# data description says NA means "no fence", as given that the huge ratio of missing value,
# so we are going to fill null values with none
train_test_data["Fence"] = train_test_data["Fence"].fillna("None")

In [ ]:
# now we are filling null values of column FireplaceQu
# data description says NA means "no fireplace",as given that the huge ratio of missing value,
# so we are going to fill null values with none
train_test_data["FireplaceQu"] = train_test_data["FireplaceQu"].fillna("None")

In [ ]:
# now we are filling null values of column LotFrontage
# we are going to Group by neighborhood and fill in missing value 
# by the median LotFrontage of all the neighborhood
train_test_data["LotFrontage"] = train_test_data.groupby("Neighborhood")["LotFrontage"].transform(
    lambda x: x.fillna(x.median()))

In [ ]:
# now we are filling null values of columns GarageYrBlt, GarageArea and GarageCars  
# Replacing missing data with 0 Since No garage = no cars in such garage.
train_test_data["GarageYrBlt"] = train_test_data["GarageYrBlt"].fillna(0)
train_test_data["GarageArea"] = train_test_data["GarageArea"].fillna(0)
train_test_data["GarageCars"] = train_test_data["GarageCars"].fillna(0)



In [ ]:
# now we are filling null values of columns GarageType, GarageFinish, GarageQual and GarageCond 
# replacing missing data with none 
train_test_data["GarageType"] = train_test_data["GarageType"].fillna("None")
train_test_data["GarageFinish"] = train_test_data["GarageFinish"].fillna("None")
train_test_data["GarageQual"] = train_test_data["GarageQual"].fillna("None")
train_test_data["GarageCond"] = train_test_data["GarageCond"].fillna("None")

In [ ]:
# now we are filling null values of columns BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, BsmtFullBath 
# and BsmtHalfBath
# Replacing missing values with 0 for having no basement
train_test_data["BsmtFinSF1"] = train_test_data["BsmtFinSF1"].fillna(0)
train_test_data["BsmtFinSF2"] = train_test_data["BsmtFinSF2"].fillna(0)
train_test_data["BsmtUnfSF"] = train_test_data["BsmtUnfSF"].fillna(0)
train_test_data["TotalBsmtSF"] = train_test_data["TotalBsmtSF"].fillna(0)
train_test_data["BsmtFullBath"] = train_test_data["BsmtFullBath"].fillna(0)
train_test_data["BsmtHalfBath"] = train_test_data["BsmtHalfBath"].fillna(0)


In [ ]:
# now we are filling null values in these categorical columns
# BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1 and BsmtFinType2
# replacing missing values with none for having on basement
train_test_data["BsmtQual"] = train_test_data["BsmtQual"].fillna("none")
train_test_data["BsmtCond"] = train_test_data["BsmtCond"].fillna("none")
train_test_data["BsmtExposure"] = train_test_data["BsmtExposure"].fillna("none")
train_test_data["BsmtFinType1"] = train_test_data["BsmtFinType1"].fillna("none")
train_test_data["BsmtFinType2"] = train_test_data["BsmtFinType2"].fillna("none")

In [ ]:
# now we are filling null values of columns MasVnrArea and MasVnrType
# replacing missing value with none in column MasVnrType and 0 in column MasVnrArea 
# for having no masonry veneer for these houses
train_test_data["MasVnrArea"] = train_test_data["MasVnrArea"].fillna(0)
train_test_data["MasVnrType"] = train_test_data["MasVnrType"].fillna("None")

In [ ]:
# now we are filling null values of column MSZoning
# as 'RL' is by far the most common value.
# so we are replacing missing values with RL using mode
train_test_data["MSZoning"] = train_test_data["MSZoning"].fillna(train_test_data['MSZoning'].mode()[0])

In [ ]:
# here we are removing Utilities column because it will not play important role in doing prediction
train_test_data = train_test_data.drop(['Utilities'], axis=1)

In [ ]:
# here we are filling null values of column Functional
# data description says NA means typical so we are filling missing values with TYP
train_test_data["Functional"] = train_test_data["Functional"].fillna("Typ")

In [ ]:
# here we are filling null values of column Electrical
# It has one NA value. Since this feature has mostly 'SBrkr', 
# so we are filling missing values with "SBrkr" using mode
train_test_data["Electrical"] = train_test_data["Electrical"].fillna(train_test_data["Electrical"].mode()[0])

In [ ]:
# here we are filling null values of column KitchenQual  
# Only one NA value, and same as Electrical,since this feature has mostly "TA",
# so we are filling missing values with "TA" using mode
train_test_data["KitchenQual"] = train_test_data["KitchenQual"].fillna(train_test_data["KitchenQual"].mode()[0])

In [ ]:
# here we are filling null values of columns Exterior1st and Exterior2nd
# Again Both Exterior 1 & 2 have only one missing value,
# so we are filling missing values using mode
train_test_data["Exterior1st"] = train_test_data["Exterior1st"].fillna(train_test_data["Exterior1st"].mode()[0])
train_test_data["Exterior2nd"] = train_test_data["Exterior2nd"].fillna(train_test_data["Exterior2nd"].mode()[0])

In [ ]:
# here we are filling null values of columns SaleType
# Since this feature has mostly 'WD'
# so we are filling missing values with "WD" using mode
train_test_data["SaleType"] = train_test_data["SaleType"].fillna(train_test_data["SaleType"].mode()[0])

In [ ]:
# here we are filling null values of columns MSSubClass
# data discription Na most likely means No building class
# so we are going to replace missing values with none
train_test_data["MSSubClass"] = train_test_data["MSSubClass"].fillna("none")

In [ ]:
# now checking ones again if there is any missing value or not
all_data_na = (train_test_data.isnull().sum() / len(train_test_data)) * 100
all_data_na = all_data_na.drop(all_data_na[all_data_na == 0].index).sort_values(ascending=False)[:30]
missing_data = pd.DataFrame({'Missing Ratio' :all_data_na})
missing_data.head()

**Transforming some numerical variables that are really categorical**

In [ ]:
#MSSubClass The building class
train_test_data['MSSubClass'] = train_test_data['MSSubClass'].apply(str)


#Changing OverallCond into a categorical variable
train_test_data['OverallCond'] = train_test_data['OverallCond'].astype(str)


#Changing Year and month sold are into categorical features.
train_test_data['YrSold'] = train_test_data['YrSold'].astype(str)
train_test_data['MoSold'] = train_test_data['MoSold'].astype(str)

**Performing Label Encoding**

In [ ]:
# here we are imporitng libraries for performing label encoder
from sklearn.preprocessing import LabelEncoder
columns = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold')
for c in columns:
    lb = LabelEncoder()
    lb.fit(list(train_test_data[c].values))
    train_test_data[c] = lb.transform(list(train_test_data[c].values))
# here we are printing shape of our dataset
train_test_data.shape

In [ ]:
# here we are adding one more feature
# adding total sqfootage feature this feature is combination of total area of basement,first
# and second floor area of each house
train_test_data["TotalSF"] = train_test_data["TotalBsmtSF"]+train_test_data["1stFlrSF"]+train_test_data["2ndFlrSF"]

**Skewed features**

In [ ]:
from scipy import stats
from scipy.stats import norm, skew
numeric_feats = train_test_data.dtypes[train_test_data.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = train_test_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)

**Box Cox Transformation of (highly) skewed features**

In [ ]:
skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    #all_data[feat] += 1
    train_test_data[feat] = boxcox1p(train_test_data[feat], lam)
    
#all_data[skewed_features] = np.log1p(all_data[skewed_features])

In [ ]:
# creating dummy variable of categorical features
train_test_data = pd.get_dummies(train_test_data)
print(train_test_data.shape)

# Building Model

In [ ]:
# here we are getting new set of train and test
x = train_test_data[:ntrain]
y = train_test_data[ntrain:]

In [ ]:
# here we are imporitng important libraries which we are needed in doing prediction
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge,LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [ ]:
#Validation function
n_folds = 5

def rmsle_cv(model):
    kf = KFold(n_folds, shuffle=True, random_state=42).get_n_splits(x.values)
    rmse= np.sqrt(-cross_val_score(model, x.values, y_train, scoring="neg_mean_squared_error", cv = kf))
    return(rmse)

#  Base Models

**Lasso regrssion**

This model may be very sensitive to outliers. So we need to made it more robust on them. For that we use the sklearn's Robustscaler() method on pipeline.

In [ ]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))

In [ ]:
# here we are calculating base model score
score = rmsle_cv(lasso)
print("\nLasso score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))


**Elastic Net Regression**

again for outliers

In [ ]:
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))

In [ ]:
# calculating Base model score
score = rmsle_cv(ENet)
print("ElasticNet score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

**Kernel Ridge Regression**

In [ ]:
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)

In [ ]:
# calculating base model score
score = rmsle_cv(KRR)
print("Kernel Ridge score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

**LightGBM**

In [ ]:
model_lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)


In [ ]:
# calculating base model score
score = rmsle_cv(model_lgb)
print("LGBM score: {:.4f} ({:.4f})\n" .format(score.mean(), score.std()))

# Stacking models

**Averaged base models class**

In [ ]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)   

**Averaged base models score**

In [ ]:
averaged_models = AveragingModels(models = (ENet, KRR, lasso))

score = rmsle_cv(averaged_models)
print(" Averaged base models score: {:.4f} ({:.4f})\n".format(score.mean(), score.std()))

**Stacking averaged Models Class**

In [ ]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=5):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                instance.fit(X[train_index], y[train_index])
                y_pred = instance.predict(X[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)


**Stacking Averaged models Score**

In [ ]:
stacked_averaged_models = StackingAveragedModels(base_models = (ENet,KRR),
                                                 meta_model = lasso)

score = rmsle_cv(stacked_averaged_models)
print("Stacking Averaged models score: {:.4f} ({:.4f})".format(score.mean(), score.std()))

In [ ]:
def rmsle(Y, y_pred):
    return np.sqrt(mean_squared_error(Y, y_pred))

**Final Training and Prediction**

In [ ]:
stacked_averaged_models.fit(x.values, y_train)
stacked_train_pred = stacked_averaged_models.predict(x.values)
stacked_pred = np.expm1(stacked_averaged_models.predict(y.values))
print(rmsle(y_train, stacked_train_pred))

In [ ]:
model_lgb.fit(x, y_train)
lgb_train_pred = model_lgb.predict(x)
lgb_pred = np.expm1(model_lgb.predict(y.values))
print(rmsle(y_train, lgb_train_pred))

In [ ]:
# RMSE on the entire Train data when averaging

print('RMSLE score on train data:')
print(rmsle(y_train,stacked_train_pred*0.70 +
               + lgb_train_pred*0.15 ))

In [ ]:
ensemble = stacked_pred*0.70+ lgb_pred*0.15
ensemble

**Submission**

In [ ]:
sub = pd.DataFrame()
sub['Id'] = test_ID
sub['SalePrice'] = ensemble
sub.to_csv('submission.csv',index=False)
print(sub)

# Conclusion

Here we have completed our prediction . If there is need of any improvement that please feel free to share it.
If you found this notebook helpful or you just liked it , some upvotes would be very much appreciated That will keep me motivated to update it on a regular basis.